In [ ]:
"""
- VGG19 siamese train
    - save results to cifar_vgg/models/Cifar10-vgg-siamese-2-400ep-90proc.h5
"""

In [1]:
import tensorflow as tf
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras import *
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
import random
import utils
import sys
from utils import *

2024-04-28 20:29:27.504073: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-28 20:29:27.534384: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 20:29:27.976840: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Y_train = y_train
Y_test = y_test


num_classes = 10 
x_train = x_train/255.
y_train = to_categorical(y_train, num_classes) 

x_test = x_test/255.
y_test = to_categorical(y_test, num_classes) 

In [3]:
##########################################Functions to rename vgg19 model ###################################

def add_infix(a,b,character):  
    b.startswith(a)
    tail = b.split(a, 1)[1]
    b = a + character + tail
    return b

add_infix('block1_conv1','block1_conv1_ib-0','A')

def rename_network_nodes(vgg19,character):  # https://stackoverflow.com/questions/63373692/how-to-rename-the-layers-of-a-keras-model-without-corrupting-the-structure
    layer_names = {layer._name for layer in vgg19.layers}
    _network_nodes = []
    for b in vgg19._network_nodes:
        for a in layer_names:
            if(b.startswith(a)):
                _network_nodes.append(add_infix(a,b,character))
    vgg19._network_nodes = set(_network_nodes)
    
def rename(vgg19,character):        
    rename_network_nodes(vgg19,character)
    
    vgg19._name = vgg19._name + f"{character}"
    for layer in vgg19.layers:
        layer._name = layer._name + f"{character}"

######################################################################################################

In [4]:
def build_model(weight_decay= 0.0005, x_shape=[32,32,3], num_classes=10):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

    model = Sequential()
#     model.add(Input(shape=()))

    model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

#     model.add(Dropout(0.5))
#     model.add(Dense(num_classes))
#     model.add(Activation('softmax'))
    return model

In [5]:
########################################## Building siamese network ###################################


def build_siamese_vgg_model(shape, class_num):
    inputs1 = Input(shape)
    inputs2 = Input(shape)
    

    x1 = build_model()
    x2 = build_model()

    x = layers.concatenate([x1.outputs[0], x2.outputs[0]])
    
    outputs = layers.Dense(num_classes, activation='softmax')(x)  

    # build the model
    model = Model([x1.inputs[0], x2.inputs[0]], outputs)
    
    model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model

model = build_siamese_vgg_model(x_train[0].shape, num_classes)

print(model.summary())

/home/x/miniconda3/envs/ml4/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-04-28 20:29:29.534401: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9777 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional_115"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │      1,792 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 32, 32,    │      1,792 │ input_layer_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 32,    │          0 │ conv2d[56][0]     │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_14       │ (None, 32, 32,    │          0 │ conv2d_13[56][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │        256 │ activation[55][0] │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ activation_14[55… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │     36,928 │ dropout[53][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 32, 32,    │     36,928 │ dropout_9[53][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 32,    │          0 │ conv2d_1[52][0]   │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_15       │ (None, 32, 32,    │          0 │ conv2d_14[52][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ activation_1[51]… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ activation_15[51… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 16, 16,    │          0 │ batch_normalizat

 Total params: 30,002,826 (114.45 MB)

 Trainable params: 29,983,882 (114.38 MB)

 Non-trainable params: 18,944 (74.00 KB)

None


In [6]:
# x2.summary()

In [7]:
# x1 = build_model()
# # print(x1.summary())
# x2 = build_model()
# # print(x2.summary())

# x = layers.concatenate([x1.outputs[0], x2.outputs[0]])

In [8]:
def make_pairs(images, labels):
    pairImages = []
    pairLabels = []

    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]

    for idxA in range(len(images)):
        currentImage = images[idxA]
        label = labels[idxA]
#         print(currentImage.shape, label)

        
        for idxB in random.sample(list(idx[label[0]]), 1):
            posImage = images[idxB]
#             print(posImage.shape)
#             print('\n\n\n')


            pairImages.append(np.array([currentImage, posImage]))
            pairLabels.append(np.array(label))
        
    return np.array(pairImages), np.array(pairLabels)

In [9]:
######################################################################################################

x_train_siamese, y_train_siamese = make_pairs(x_train, Y_train)
x_test_siamese, y_test_siamese = make_pairs(x_test, Y_test)

y_train_siamese = to_categorical(y_train_siamese, num_classes) 
y_test_siamese = to_categorical(y_test_siamese, num_classes) 

x_train_both = [x_train_siamese[:,0,:], x_train_siamese[:,1,:]]
x_test_both = [x_test_siamese[:,0,:], x_test_siamese[:,1,:]]


print('Images')
print(np.array(x_train_both).shape)
print(np.array(x_test_both).shape)


print('Labels')
print(y_train_siamese.shape)
print(y_test_siamese.shape)

Images
(2, 50000, 32, 32, 3)
(2, 10000, 32, 32, 3)
Labels
(50000, 10)
(10000, 10)


In [10]:


# print(model.summary())

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)

history3 = model.fit(x_train_both, y_train_siamese, epochs=400, batch_size=512, shuffle=True, validation_data=(x_test_both, y_test_siamese),callbacks=[callback])


# print('Saving model')
# model.save('cifar_vgg/models/model.h5')

Epoch 1/400


I0000 00:00:1714328982.285175  137929 service.cc:145] XLA service 0x7e2480001c50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714328982.285201  137929 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-04-28 20:29:42.659398: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-28 20:29:43.132662: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1714329020.910614  137929 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


98/98 ━━━━━━━━━━━━━━━━━━━━ 100s 597ms/step - accuracy: 0.1225 - loss: 7.2679 - val_accuracy: 0.1000 - val_loss: 6.9623
Epoch 2/400
98/98 ━━━━━━━━━━━━━━━━━━━━ 24s 250ms/step - accuracy: 0.2024 - loss: 6.6897 - val_accuracy: 0.1009 - val_loss: 7.4732
Epoch 3/400
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 251ms/step - accuracy: 0.2711 - loss: 6.3647 - val_accuracy: 0.1082 - val_loss: 8.0621
Epoch 4/400
98/98 ━━━━━━━━━━━━━━━━━━━━ 29s 293ms/step - accuracy: 0.3230 - loss: 6.1759 - val_accuracy: 0.1250 - val_loss: 8.6893
Epoch 5/400
98/98 ━━━━━━━━━━━━━━━━━━━━ 28s 284ms/step - accuracy: 0.3618 - loss: 6.0317 - val_accuracy: 0.1365 - val_loss: 8.2644
Epoch 6/400
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 258ms/step - accuracy: 0.3921 - loss: 5.9325 - val_accuracy: 0.1553 - val_loss: 7.9985
Epoch 7/400
98/98 ━━━━━━━━━━━━━━━━━━━━ 26s 264ms/step - accuracy: 0.4226 - loss: 5.8368 - val_accuracy: 0.2142 - val_loss: 7.2699
Epoch 8/400
98/98 ━━━━━━━━━━━━━━━━━━━━ 25s 257ms/step - accuracy: 0.4506 - loss: 5.7585 - val_accurac

In [17]:
model.save('cifar_vgg/models/Cifar10-vgg-siamese-2-400ep-90proc.h5')

In [18]:
model = tf.keras.models.load_model("cifar_vgg/models/Cifar10-vgg-siamese-2-400ep-90proc.h5")

results = full_evaluate(model, x_test_both, Y_test)

print()



313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step
accuracy_score: 0.9019
f1_score_macro: 0.9024842808375159
f1_score_weighted: 0.9013157191624841



: 